## Introdution
There are many ways data can end up with missing values. For example,

A 2 bedroom house won't include a value for the size of a third bedroom.
A survey respondent may choose not to share his income.
Most machine learning libraries (including scikit-learn) give an error if you try to build a model using data with missing values. So you'll need to choose one of the strategies below.

## Three Approaches
### 1) A Simple Option: Drop Columns with Missing Values
The simplest option is to drop columns with missing values.

tut2_approach1

Unless most values in the dropped columns are missing, the model loses access to a lot of (potentially useful!) information with this approach. As an extreme example, consider a dataset with 10,000 rows, where one important column is missing a single entry. This approach would drop the column entirely!

### 2) A Better Option: Imputation
Imputation fills in the missing values with some number. For instance, we can fill in the mean value along each column.

tut2_approach2

The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would get from dropping the column entirely.

### 3) An Extension To Imputation
Imputation is the standard approach, and it usually works well. However, imputed values may be systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.

tut3_approach3

In this approach, we impute the missing values, as before. And, additionally, for each column with missing entries in the original dataset, we add a new column that shows the location of the imputed entries.

In some cases, this will meaningfully improve results. In other cases, it doesn't help at all.

## Example
In the example, we will work with the Melbourne Housing dataset. Our model will use information such as the number of rooms and land size to predict home price.

We won't focus on the data loading step. Instead, you can imagine you are at a point where you already have the training and validation data in X_train, X_valid, y_train, and y_valid.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('D:\ML\melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [4]:
X

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.79960,144.99840,4019.0
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.80790,144.99340,4019.0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.80930,144.99440,4019.0
3,3,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.79690,144.99690,4019.0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.80720,144.99410,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13575,4,16.7,3150.0,4.0,2.0,2.0,652.0,NaN,1981.0,-37.90562,145.16761,7392.0
13576,3,6.8,3016.0,3.0,2.0,2.0,333.0,133.0,1995.0,-37.85927,144.87904,6380.0
13577,3,6.8,3016.0,3.0,2.0,4.0,436.0,NaN,1997.0,-37.85274,144.88738,6380.0
13578,4,6.8,3016.0,4.0,1.0,5.0,866.0,157.0,1920.0,-37.85908,144.89299,6380.0


## Score from Approach 1 (Drop Columns with Missing Values)
Since we are working with both training and validation sets, we are careful to drop the same columns in both DataFrames.



In [6]:
cols_with_missing=[col for col in X_train.columns if X_train[col].isnull().any()]

In [9]:
reduced_X_train =X_train.drop(cols_with_missing,axis=1)
reduced_X_valid=X_valid.drop(cols_with_missing,axis=1)
print("MAE from Approach  1 (DROP COLUMNS WITH MISSING VALUES):")
print(score_dataset(reduced_X_train,reduced_X_valid,y_train,y_valid))

MAE from Approach  1 (DROP COLUMNS WITH MISSING VALUES):
183550.22137772635


## Score from Approach 2 (Imputation)
Next, we use SimpleImputer to replace missing values with the mean value along each column.

Although it's simple, filling in the mean value generally performs quite well (but this varies by dataset). While statisticians have experimented with more complex ways to determine imputed values (such as regression imputation, for instance), the complex strategies typically give no additional benefit once you plug the results into sophisticated machine learning models.

In [34]:
from sklearn.impute import SimpleImputer
#Imputer
my_imputer=SimpleImputer(strategy='median')
imputer_X_train=pd.DataFrame(my_imputer.fit_transform(X_train))
imputer_X_valid=pd.DataFrame(my_imputer.transform(X_valid))

imputer_X_train


,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,5.0,3182.0,1.0,1.0,1.0,0.0,126.0,1940.0,-37.85984,144.98670,13240.0
1,2.0,8.0,3016.0,2.0,2.0,1.0,193.0,126.0,1970.0,-37.85800,144.90050,6380.0
2,3.0,12.6,3020.0,3.0,1.0,1.0,555.0,126.0,1970.0,-37.79880,144.82200,3755.0
3,3.0,13.0,3046.0,3.0,1.0,1.0,265.0,126.0,1995.0,-37.70830,144.91580,8870.0
4,3.0,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10859,3.0,5.2,3056.0,3.0,1.0,2.0,212.0,126.0,1970.0,-37.77695,144.95785,11918.0
10860,3.0,10.5,3081.0,3.0,1.0,1.0,748.0,101.0,1950.0,-37.74160,145.04810,2947.0
10861,4.0,6.7,3058.0,4.0,2.0,2.0,441.0,255.0,2002.0,-37.73572,144.97256,11204.0
10862,3.0,12.0,3073.0,3.0,1.0,1.0,606.0,126.0,1970.0,-37.72057,145.02615,21650.0


In [35]:
imputer_X_valid

,0,1,2,3,4,5,6,7,8,9,10,11
0,4.0,8.0,3016.0,4.0,2.0,2.0,450.0,190.0,1910.0,-37.86100,144.89850,6380.0
1,2.0,6.6,3011.0,2.0,1.0,0.0,172.0,81.0,1900.0,-37.81000,144.88960,2417.0
2,3.0,10.5,3020.0,3.0,1.0,1.0,581.0,126.0,1970.0,-37.76740,144.82421,4217.0
3,3.0,4.5,3181.0,2.0,2.0,1.0,128.0,134.0,2000.0,-37.85260,145.00710,7717.0
4,3.0,8.5,3044.0,3.0,2.0,2.0,480.0,126.0,1970.0,-37.72523,144.94567,7485.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2.0,6.4,3011.0,2.0,1.0,1.0,47.0,35.0,2013.0,-37.80140,144.89590,7570.0
2712,4.0,8.0,3016.0,4.0,2.0,4.0,551.0,126.0,1970.0,-37.85790,144.87860,6380.0
2713,3.0,10.8,3105.0,3.0,1.0,1.0,757.0,126.0,1970.0,-37.78094,145.10131,4480.0
2714,4.0,6.2,3039.0,4.0,1.0,3.0,478.0,152.0,1925.0,-37.76421,144.90571,6232.0


In [36]:
#Imputer removed column names put them back
imputer_X_train.columns=X_train.columns
imputer_X_valid.columns=X_valid.columns
imputer_X_train

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,1.0,5.0,3182.0,1.0,1.0,1.0,0.0,126.0,1940.0,-37.85984,144.98670,13240.0
1,2.0,8.0,3016.0,2.0,2.0,1.0,193.0,126.0,1970.0,-37.85800,144.90050,6380.0
2,3.0,12.6,3020.0,3.0,1.0,1.0,555.0,126.0,1970.0,-37.79880,144.82200,3755.0
3,3.0,13.0,3046.0,3.0,1.0,1.0,265.0,126.0,1995.0,-37.70830,144.91580,8870.0
4,3.0,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10859,3.0,5.2,3056.0,3.0,1.0,2.0,212.0,126.0,1970.0,-37.77695,144.95785,11918.0
10860,3.0,10.5,3081.0,3.0,1.0,1.0,748.0,101.0,1950.0,-37.74160,145.04810,2947.0
10861,4.0,6.7,3058.0,4.0,2.0,2.0,441.0,255.0,2002.0,-37.73572,144.97256,11204.0
10862,3.0,12.0,3073.0,3.0,1.0,1.0,606.0,126.0,1970.0,-37.72057,145.02615,21650.0


In [37]:
print("MAE from Approach 2(Imputation):")
print(score_dataset(imputer_X_train,imputer_X_valid,y_train,y_valid))

MAE from Approach 2(Imputation):
178045.2412125675


## Score from Approach 3 (An Extension to Imputation)
Next, we impute the missing values, while also keeping track of which values were imputed.

In [18]:
# make copy to avoid changing original data (when imputing)
X_train_plus =X_train.copy()
X_valid_plus =X_valid.copy()

#make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col+'_was_missing']=X_train_plus[col].isnull()
    X_valid_plus[col+'_was_missing']=X_valid_plus[col].isnull()
X_train_plus

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car_was_missing,BuildingArea_was_missing,YearBuilt_was_missing
12167,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.98670,13240.0,False,True,False
6524,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.90050,6380.0,False,True,True
8413,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.82200,3755.0,False,True,True
2919,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.91580,8870.0,False,True,False
6043,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.82720,4217.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,3,5.2,3056.0,3.0,1.0,2.0,212.0,NaN,NaN,-37.77695,144.95785,11918.0,False,True,True
3264,3,10.5,3081.0,3.0,1.0,1.0,748.0,101.0,1950.0,-37.74160,145.04810,2947.0,False,False,False
9845,4,6.7,3058.0,4.0,2.0,2.0,441.0,255.0,2002.0,-37.73572,144.97256,11204.0,False,False,False
10799,3,12.0,3073.0,3.0,1.0,1.0,606.0,NaN,NaN,-37.72057,145.02615,21650.0,False,True,True


In [32]:
#Imputation
my_imputer=SimpleImputer(strategy='median')
imputed_X_train_plus=pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus=pd.DataFrame(my_imputer.transform(X_valid_plus))

#Imputation removed columns names put them back
imputed_X_train_plus.columns=X_train_plus.columns
imputed_X_valid_plus.columns=X_valid_plus.columns

In [33]:
print("MAE FROM Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus,imputed_X_valid_plus,y_train,y_valid))

MAE FROM Approach 3 (An Extension to Imputation):
179669.92839960728


## So, why did imputation perform better than dropping the columns?
The training data has 10864 rows and 12 columns, where three columns contain missing data. For each column, less than half of the entries are missing. Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.

In [31]:
print(X_train.shape)
missing_val_count_by_column = (X_train.isnull().sum())
missing_val_count_by_column

(10864, 12)


Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                49
Landsize            0
BuildingArea     5156
YearBuilt        4307
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64

In [23]:
print(missing_val_count_by_column[missing_val_count_by_column>0])

Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
